# 租车问题 1

## 目标和先决条件

通过这个示例提高您的建模技能,您将学习如何使用数学优化来确定租车公司应该拥有多少辆车,以及每天应该将它们放在哪里以实现每周利润最大化。

此模型是H. Paul Williams的《数学规划模型构建》第五版第284-286页和340-342页的示例25。

这是一个中级示例,我们假设您了解Python和Gurobi Python API,并且对构建数学优化模型有一定了解。

**下载代码库** <br />
您可以点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的代码库。

---
## 问题描述

一家小型租车公司(只租赁一种类型的汽车)在格拉斯哥、曼彻斯特、伯明翰和普利茅斯设有服务网点。除周日(公司休息)外,每个工作日都有预计需求。这些估计数据如下表所示。不必满足所有需求。

![weeklyDemand](weeklyDemand.PNG)


汽车可以租用一天、两天或三天,并在第二天早上返还到租车的网点或其他网点。例如,周四的2天租期意味着汽车必须在周六早上返还;周五的3天租期意味着汽车必须在周二早上返还。周六的1天租期意味着汽车必须在周一早上返还,星期二早上为2天租期。

租期与出发地和目的地无关。根据历史数据,公司知道租期的分布情况:55%的汽车租用一天,20%租用两天,25%租用三天。目前估计从每个网点租出并返回到给定网点的汽车百分比(与日期无关)如下表所示。

![FromToPct](FromToPct.PNG)

公司租出汽车的边际成本('磨损'、管理等)估计如下:

| 租期 | 边际成本 |
| --- | --- |
| 1天 | $\$ 20$ |
| 2天 | $\$ 25$ |
| 3天 | $\$ 30$ |

拥有一辆车的'机会成本'(资本利息、存储、维修等)为每周$\$ 15$。

可以将未损坏的汽车从一个网点转移到另一个网点,不考虑距离。在转移的当天,汽车不能出租。每辆车的转移成本(单位:美元)如下表所示。

![FromToCst](FromToCst.PNG)

10%的由客户返还的汽车会损坏。发生这种情况时,向客户收取$\$ 100$的额外费用(无论损坏程度如何,公司都完全由保险承担)。此外,汽车必须转移到修理网点,将在第二天进行修理。损坏汽车的转移成本与未损坏的相同(除非修理网点就是当前网点,这种情况下成本为$\$0$)。损坏汽车的转移需要一天时间,除非已经在修理网点。
到达修理网点后,所有类型的修理(或更换)都需要一天时间。只有两个网点有修理能力。每个修理网点的日修理能力(辆/天)如下:

| 修理网点 | 修理能力 |
| --- | --- |
| 曼彻斯特 | 12 |
| 伯明翰 | 20 |

修理完成后,汽车第二天可以在该网点出租或转移到其他网点(需要一天)。因此,周三早上返还损坏的汽车会在周三转移到修理网点(如果不是当前网点),周四修理,周五早上可以在修理网点出租。
租金取决于租期以及是否返回同一网点。价格(单位:美元)如下表所示。

![RentalPrice](RentalPrice.PNG)

我们假设每天开始时:
1. 客户归还到期的汽车。
2. 损坏的汽车被送到修理网点。
3. 从其他网点转移的汽车到达。
4. 发出转移。
5. 汽车出租。
6. 如果是修理网点,则修理完的汽车可供出租。

目标是确定租车公司应该拥有多少辆车,以及为了最大化每周利润,这些车在每个工作日开始时应该放在哪里。公司希望得到一个'稳态'解决方案,在后续几周的相同日期,每个网点都有相同的预期车辆数量。

---
## 模型构建

$d,d2 \in \text{Depots}=\{\text{Glasgow}, \text{Manchester}, \text{Birmingham},  \text{Plymouth}\}$

$\text{NRD}=\{\text{Glasgow}, \text{Plymouth}\}$: 没有修理能力的网点。

$\text{RD}=\{\text{Manchester}, \text{Birmingham}\}$: 有修理能力的网点。

$t \in \text{Days}=\{\text{周一},\text{周二},\text{周三},\text{周四},\text{周五},\text{周六}\}$

$r \in \text{RentDays}=\{1,2,3\}$: 租赁天数。

### 参数

$\text{demand}_{d,t} \in \mathbb{R}^+$: 网点 $d$ 在 $t$ 日的预计租赁需求。

$\text{pctDepot}_{d,d2} \in \mathbb{R}^+$: 从网点 $d$ 租出并返回至网点 $d2$ 的汽车比例。

$\text{cstTransfer}_{d,d2} \in \mathbb{R}^+$: 从网点 $d$ 向网点 $d2$ 转移一辆车的成本。

$\text{pctRent}_{r} \in \mathbb{R}^+$: 租期为 $r$ 天的汽车比例。

$\text{capRepair}_{d} \in \mathbb{R}^+$: 网点 $d$ 的修理能力。

$\text{cstSameDepot}_{r} \in \mathbb{R}^+$: 租期 $r$ 天且返回同一网点的租金。

$\text{cstOtherDepot}_{r} \in \mathbb{R}^+$: 租期 $r$ 天且返回其他网点的租金。

$\text{marginalCost}_{r} \in \mathbb{R}^+$: 公司租出一辆车 $r$ 天的边际成本。

$\text{pctUndamaged } \in [0,1]$: 客户返还的未损坏汽车百分比。

$\text{pctDamaged  } \in [0,1]$: 客户返还的损坏汽车百分比。

$\text{cstOwn} \in \mathbb{R}^+$: 拥有一辆车的成本。

$\text{damagedFee} = 10$: 损坏车辆费用。10%的汽车会损坏,每辆损坏车的费用是$\$100$。

### 决策变量

$\text{xOwned} \in \mathbb{R}^+$: 拥有的汽车总数。

$\text{xUndamaged}_{d,t} \in \mathbb{R}^+$: $t$ 日开始时网点 $d$ 可用的未损坏汽车数量。

$\text{xDamaged}_{d,t} \in \mathbb{R}^+$: $t$ 日开始时网点 $d$ 可用的损坏汽车数量。

$\text{xRented}_{d,t} \in \mathbb{R}^+$: $t$ 日开始时从网点 $d$ 租出的汽车数量。

$\text{xUDleft}_{d,t} \in \mathbb{R}^+$: $t$ 日开始时网点 $d$ 可用的未损坏汽车数量。

$\text{xDleft}_{d,t} \in \mathbb{R}^+$: $t$ 日结束时网点 $d$ 剩余的损坏汽车数量。

$\text{xUDtransfer}_{d,d2,t} \in \mathbb{R}^+$: $t$ 日开始时网点 $d$ 待转移至网点 $d2$ 的未损坏汽车数量。

$\text{xDtransfer}_{d,d2,t} \in \mathbb{R}^+$: $t$ 日开始时网点 $d$ 待转移至网点 $d2$ 的损坏汽车数量。

$\text{xRepaired}_{d,t} \in \mathbb{R}^+$: $t$ 日期间在网点 $d$ 需要修理的损坏汽车数量。

### 目标函数
目标是最大化利润。

\begin{equation}
\sum_{d \in \text{Depots}}
\sum_{t \in \text{Days}}
\sum_{r \in \text{RentDays}} 
\text{pctDepot}_{d,d}*\text{pctRent}_{r}*(\text{cstSameDepot}_{r} - \text{marginalCost}_{r} + \text{damagedFee})*\text{xRented}_{d,t}
\end{equation}

\begin{equation}
+ \sum_{d \in \text{Depots}} \sum_{d2 \in \text{Depots}}
\sum_{t \in \text{Days}}
\sum_{r \in \text{RentDays}} 
\text{pctDepot}_{d,d2}*\text{pctRent}_{r}*(\text{cstOtherDepot}_{r} - \text{marginalCost}_{r} + \text{damagedFee})*\text{xRented}_{d,t}
\end{equation}

\begin{equation}
- \sum_{d \in \text{Depots}} \sum_{d2 \in \text{Depots}}
\sum_{t \in \text{Days}} \text{cstTransfer}_{d,d2}*(\text{xUDtransfer}_{d,d2,t} + \text{xDtransfer}_{d,d2,t} )
- \text{cstOwn}*\text{xOwned}
\end{equation}


### 约束条件

**非修理网点的未损坏汽车** <br />
$t$ 日开始时非修理网点 $d$ 可用的未损坏汽车数量。

\begin{equation}
\sum_{d2 \in \text{Depots}} 
\sum_{r \in \text{RentDays}} \text{pctUndamaged}*\text{pctDepot}_{d2,d}*\text{pctRent}_{r}*\text{xRented}_{d2,(t-r)mod(6)}
\end{equation}

\begin{equation}
+ \sum_{d2 \in \text{Depots}} \text{xUDtransfer}_{d2,d,(t-1)mod(6)} + \text{xUDleft}_{d,(t-1)mod(6)} = \text{xUndamaged}_{d,t} 
\quad \forall d \in NRD, t \in Days
\end{equation}

非修理网点 $d$ 在 $t$ 日的未损坏汽车需求。

\begin{equation}
\text{xUndamaged}_{d,t} = \text{xRented}_{d,t} + 
\sum_{d2 \in \text{Depots}} \text{xUDtransfer}_{d,d2,t} + \text{xUDleft}_{d,t}
\quad \forall d \in NRD, t \in Days
\end{equation}

**修理网点的未损坏汽车** <br />
$t$ 日开始时修理网点 $d$ 可用的未损坏汽车数量。

\begin{equation}
\sum_{d2 \in \text{Depots}} 
\sum_{r \in \text{RentDays}} \text{pctUndamaged}*\text{pctDepot}_{d2,d}*\text{pctRent}_{r}*\text{xRented}_{d2,(t-r)mod(6)}
\end{equation}

\begin{equation}
+ \sum_{d2 \in \text{Depots}} \text{xUDtransfer}_{d2,d,(t-1)mod(6)} 
+ \text{xRepaired}_{d,(t-1)mod(6)}  + \text{xUDleft}_{d,(t-1)mod(6)} = \text{xUndamaged}_{d,t} 
\quad \forall d \in NRD, t \in Days
\end{equation}

修理网点 $d$ 在 $t$ 日的未损坏汽车需求。

\begin{equation}
\text{xUndamaged}_{d,t} = \text{xRented}_{d,t} + 
\sum_{d2 \in \text{Depots}} \text{xUDtransfer}_{d,d2,t} + \text{xUDleft}_{d,t}
\quad \forall d \in NRD, t \in Days
\end{equation}

**非修理网点的损坏汽车** <br />
$t$ 日开始时非修理网点 $d$ 可用的损坏汽车数量。

\begin{equation}
\sum_{d2 \in \text{Depots}} 
\sum_{r \in \text{RentDays}} \text{pctDamaged}*\text{pctDepot}_{d2,d}*\text{pctRent}_{r}*\text{xRented}_{d2,(t-r)mod(6)}
\end{equation}

\begin{equation}
+ \text{xDleft}_{d,(t-1)mod(6)} = \text{xDamaged}_{d,t} \quad \forall d \in NRD, t \in Days
\end{equation}

非修理网点 $d$ 在 $t$ 日的损坏汽车需求。

\begin{equation}
\text{xDamaged}_{d,t} = 
\sum_{d2 \in \text{Depots} \cap RD} \text{xDtransfer}_{d,d2,t} + \text{xDleft}_{d,t}
\quad \forall d \in NRD, t \in Days
\end{equation}

**修理网点的损坏汽车** <br />
$t$ 日开始时修理网点 $d$ 可用的损坏汽车数量。

\begin{equation}
\sum_{d2 \in \text{Depots}} 
\sum_{r \in \text{RentDays}} \text{pctDamaged}*\text{pctDepot}_{d2,d}*\text{pctRent}_{r}*\text{xRented}_{d2,(t-r)mod(6)}
\end{equation}

\begin{equation}
+ \sum_{d2 \in \text{Depots}} \text{xDtransfer}_{d2,d,(t-1)mod(6)} 
+ \text{xDleft}_{d,(t-1)mod(6)} = \text{xdamaged}_{d,t} 
\quad \forall d \in RD, t \in Days
\end{equation}

修理网点 $d$ 在 $t$ 日的损坏汽车需求。

\begin{equation}
\text{xDamaged}_{d,t} = \text{xRepaired}_{d,t} +
\sum_{d2 \in \text{Depots} \cap RD} \text{xDtransfer}_{d,d2,t} + \text{xDleft}_{d,t}
\quad \forall d \in RD, t \in Days
\end{equation}

**网点容量** <br />
每个网点 $d$ 每天 $t$ 的修理能力。

\begin{equation}
\text{xRepaired}_{d,t} \leq \text{capRepair}_{d}
\quad \forall d \in Depots, t \in Days
\end{equation}

**网点需求** <br />
每个网点 $d$ 每天 $t$ 的需求。

\begin{equation}
\text{xRented}_{d,t} \leq \text{demand}_{d,t}
\quad \forall d \in Depots, t \in Days
\end{equation}

**汽车数量** <br />
拥有的汽车总数等于周一从所有网点租出3天的汽车数量,加上周二租出2天或3天的汽车数量,再加上周三早上在各网点的所有损坏和未损坏汽车数量。
理由: 让我们选择一天(周三),计算返还到各网点并在周三早上可用的未损坏和损坏汽车数量。让我们计算已租出但尚未返还的汽车:周一租出3天的汽车,以及周二租出2天或3天的汽车。

\begin{equation}
\sum_{d \in \text{Depots}} (0.25*\text{xRented}_{d,0} + 0.45*\text{xRented}_{d,1} + \text{xUndamaged}_{d,2}  + \text{xdamaged}_{d,2} ) = \text{xOwned}
\end{equation}

---
## Python实现

我们导入Gurobi Python模块和其他Python库。

In [ ]:
# %pip install gurobipy

In [1]:
import pandas as pd
from itertools import product

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.11 & Gurobi 11.0

## 输入数据

我们定义模型的所有输入数据。

In [ ]:
# 网点列表和每周工作日

depots = ['Glasgow','Manchester','Birmingham','Plymouth']
NRD = ['Glasgow','Plymouth'] # 无修理能力的网点
RD =['Manchester','Birmingham'] # 有修理能力的网点

days = [0,1,2,3,4,5] # 周一=0, 周二=1, ... 周六=5
rentDays = [1,2,3]

d2w, demand = gp.multidict({
    ('Glasgow',0): 100,
    ('Glasgow',1): 150,
    ('Glasgow',2): 135,
    ('Glasgow',3): 83,
    ('Glasgow',4): 120,
    ('Glasgow',5): 230,
    ('Manchester',0): 250,
    ('Manchester',1): 143,
    ('Manchester',2): 80,
    ('Manchester',3): 225,
    ('Manchester',4): 210,
    ('Manchester',5): 98,
    ('Birmingham',0): 95,
    ('Birmingham',1): 195,
    ('Birmingham',2): 242,
    ('Birmingham',3): 111,
    ('Birmingham',4): 70,
    ('Birmingham',5): 124,
    ('Plymouth',0): 160,
    ('Plymouth',1): 99,
    ('Plymouth',2): 55,
    ('Plymouth',3): 96,
    ('Plymouth',4): 115,
    ('Plymouth',5): 80
})

#修理能力
depots, capacity = gp.multidict({
    ('Glasgow'): 0,
    ('Manchester'): 12,
    ('Birmingham'): 20,
    ('Plymouth'): 0
})

# 创建字典来存储 
# pctRent: 租期为r天的汽车百分比 
# cstMarginal: 租车r天的边际成本
# prcSameD: 租车r天并返回同一网点的价格
# prcOtherD: 租车r天并返回其他网点的价格
rentDays, pctRent, costMarginal, priceSameD, priceOtherD = gp.multidict({
    (1): [0.55,20,50,70],
    (2): [0.20,25,70,100],
    (3): [0.25,30,120,150]
})

# 每周拥有一辆车的成本。
cstOwn = 15

# 损坏车辆的比例费用
damagedFee = 10

# 创建字典来存储从网点d租出并返回到网点d2的汽车比例
d2d, pctFromToD = gp.multidict({
    ('Glasgow','Glasgow'): 0.6,
    ('Glasgow','Manchester'): 0.2,
    ('Glasgow','Birmingham'): 0.1,
    ('Glasgow','Plymouth'): 0.1,
    ('Manchester','Glasgow'): 0.15,
    ('Manchester','Manchester'): 0.55,
    ('Manchester','Birmingham'): 0.25,
    ('Manchester','Plymouth'): 0.05,
    ('Birmingham','Glasgow'): 0.15,
    ('Birmingham','Manchester'): 0.2,
    ('Birmingham','Birmingham'): 0.54,
    ('Birmingham','Plymouth'): 0.11,
    ('Plymouth','Glasgow'): 0.08,
    ('Plymouth','Manchester'): 0.12,
    ('Plymouth','Birmingham'): 0.27,
    ('Plymouth','Plymouth'): 0.53
})

# 创建字典来存储汽车转移成本
d2d, cstFromToD = gp.multidict({
    ('Glasgow','Glasgow'): 0.001,
    ('Glasgow','Manchester'): 20,
    ('Glasgow','Birmingham'): 30,
    ('Glasgow','Plymouth'): 50,
    ('Manchester','Glasgow'): 20,
    ('Manchester','Manchester'): 0.001,
    ('Manchester','Birmingham'): 15,
    ('Manchester','Plymouth'): 35,
    ('Birmingham','Glasgow'): 30,
    ('Birmingham','Manchester'): 15,
    ('Birmingham','Birmingham'): 0.001,
    ('Birmingham','Plymouth'): 25,
    ('Plymouth','Glasgow'): 50,
    ('Plymouth','Manchester'): 35,
    ('Plymouth','Birmingham'): 25,
    ('Plymouth','Plymouth'): 0.001
})

# 返还的未损坏和损坏汽车的比例
pctUndamaged = 0.9
pctDamaged = 0.1


### 数据预处理
我们准备用于构建线性规划模型的数据结构。

In [ ]:
# 构建元组列表(网点,网点2),要求d!=d2

list_d2notd = []

for d,d2 in d2d:
    if (d != d2):
        tp = d,d2
        list_d2notd.append(tp)

d2notd = gp.tuplelist(list_d2notd)

# 构建元组列表(网点,网点2,天)
list_dd2t = []

for d,d2 in d2notd:
    for t in days:
        tp = d,d2,t 
        list_dd2t.append(tp)
                    
dd2t = gp.tuplelist(list_dd2t)

# 构建元组列表(网点,租期)
list_dr = []

for d in depots:
    for r in rentDays:
        tp = d,r
        list_dr.append(tp)
        
dr = gp.tuplelist(list_dr) 

# 构建元组列表(网点,天,租期 )
list_dtr = []

for d in depots:
    for t in days:
            for r in rentDays:
                tp = d,t,r
                list_dtr.append(tp)
                
dtr = gp.tuplelist(list_dtr) 

# 构建元组列表(网点,网点2,天,租期)
list_dd2tr = []

for d,d2 in d2notd:
    for t in days:
        for r in rentDays:
            tp = d,d2,t,r
            list_dd2tr.append(tp)
                    
                    
dd2tr = gp.tuplelist(list_dd2tr)

## 模型部署
我们创建一个模型和变量。主要决策变量是应拥有的汽车数量以及每周每天开始时应将它们放在哪里以实现每周利润最大化。

In [ ]:
model = gp.Model('RentalCar1')

# 拥有的汽车数量
n = model.addVar(name="cars")

# 未损坏汽车数量
nu = model.addVars(d2w, name="UDcars")

# 损坏汽车数量
nd = model.addVars(d2w, name="Dcars")

# 租出的汽车数量不能超过需求
tr = model.addVars(d2w, ub=demand, name="Hcars")
#for d,t in d2w:
    #tr[d,t].lb = 1

# 未损坏汽车的期末库存
eu = model.addVars(d2w, name="EUDcars")

# 损坏汽车的期末库存
ed = model.addVars(d2w, name="EDcars")

# 转移的未损坏汽车数量
tu = model.addVars(dd2t, name="TUDcars")

# 转移的损坏汽车数量
td = model.addVars(dd2t, name="TDcars")

# 修理的损坏汽车数量
rp = model.addVars(d2w, name="RPcars")

# 修理的损坏汽车数量不能超过网点容量
for d,t in d2w:
    rp[d,t].ub = capacity[d] #修理能力

Using license file c:\gurobi\gurobi.lic


### 约束条件
$t$ 日开始时非修理网点 $d$ 可用的未损坏汽车数量应等于该网点当天的未损坏汽车需求。

In [ ]:
# 非修理网点的未损坏汽车约束(平衡方程左侧-可用性)

UDcarsNRD_L = model.addConstrs((gp.quicksum(pctUndamaged*pctFromToD[d2,d]*pctRent[r]*tr[d2,(t-r)%6 ] for d2,r in dr ) 
                              + gp.quicksum(tu.select('*',d,(t-1)%6)  ) 
                              + eu[d,(t-1)%6 ] == nu[d,t] for d in NRD for t in days ), 
                             name="UDcarsNRD_L")

# 非修理网点的未损坏汽车约束(平衡方程右侧-需求)

UDcarsNRD_R = model.addConstrs((tr[d,t] 
                                + gp.quicksum(tu.select(d,'*',t )) 
                                + eu[d,t] == nu[d,t] for d in NRD for t in days ), name='UDcarsNRD_R' )

修理网点的未损坏汽车约束(平衡方程左侧-可用性)
修理网点的未损坏汽车约束(平衡方程右侧-需求)

In [ ]:
# 修理网点的未损坏汽车约束(平衡方程左侧-可用性)

UDcarsRD_L = model.addConstrs((gp.quicksum(pctUndamaged*pctFromToD[d2,d]*pctRent[r]*tr[d2,(t-r)%6 ] for d2,r in dr ) 
                              + gp.quicksum(tu.select('*',d,(t-1)%6)  ) + rp[d, (t-1)%6 ]
                              + eu[d,(t-1)%6 ] == nu[d,t] for d in RD for t in days ), 
                             name="UDcarsRD_L")

# 修理网点的未损坏汽车约束(平衡方程右侧-需求)

UDcarsRD_R = model.addConstrs((tr[d,t] 
                                + gp.quicksum(tu.select(d,'*',t ) ) 
                                + eu[d,t] == nu[d,t] for d in RD for t in days ), name='UDcarsRD_R' )

非修理网点的损坏汽车约束(平衡方程左侧-可用性)
非修理网点的损坏汽车约束(平衡方程右侧-需求)

In [ ]:
# 非修理网点的损坏汽车约束(平衡方程左侧-可用性)

DcarsNRD_L = model.addConstrs((gp.quicksum(pctDamaged*pctFromToD[d2,d]*pctRent[r]*tr[d2,(t-r)%6 ] for d2,r in dr ) 
                              + ed[d,(t-1)%6 ] == nd[d,t] for d in NRD for t in days ), 
                             name="DcarsNRD_L")

# 非修理网点的损坏汽车约束(平衡方程右侧-需求)

DcarsNRD_R = model.addConstrs(( gp.quicksum(td[d,d2,t] for d2 in RD ) 
                                + ed[d,t] == nd[d,t] for d in NRD for t in days ), name='DcarsNRD_R' )

修理网点的损坏汽车约束(平衡方程左侧-可用性)
修理网点的损坏汽车约束(平衡方程右侧-需求)

In [ ]:
# 修理网点的损坏汽车约束(平衡方程左侧-可用性)

DcarsRD_L = model.addConstrs((gp.quicksum(pctDamaged*pctFromToD[d2,d]*pctRent[r]*tr[d2,(t-r)%6 ] for d2,r in dr )
                              + gp.quicksum(td[d2,d,(t-1)%6 ] for d2, dd in d2notd if (dd == d)) 
                              + ed[d,(t-1)%6 ] == nd[d,t] for d in RD for t in days ), 
                             name="DcarsRD_L")

# 修理网点的损坏汽车约束(平衡方程右侧-需求)

DcarsND_R = model.addConstrs((rp[d,t] + gp.quicksum(td[d,d2,t ] for d2 in NRD ) 
                                + ed[d,t] == nd[d,t] for d in RD for t in days ), name='DcarsND_R' )

拥有的汽车总数等于周一从所有网点租出3天的汽车数量,加上周二租出2天或3天的汽车数量,再加上周三早上在各网点的所有损坏和未损坏汽车数量。

In [ ]:
# 拥有汽车总数约束
# 说明: 25%的汽车租3天,20%+25%=45%的汽车租2天或3天

carsConstr = model.addConstr((gp.quicksum(0.25*tr[d,0] + 0.45*tr[d,1] + nu[d,2] + nd[d,2] for d in depots ) 
                              == n ),name='carsConstr')

目标函数是最大化利润。

In [ ]:
# 最大化利润目标函数

model.setObjective((
    gp.quicksum(pctFromToD[d,d]*pctRent[r]*(priceSameD[r] - costMarginal[r] + damagedFee)*tr[d,t] for d,t,r in dtr )
    + gp.quicksum(pctFromToD[d,d2]*pctRent[r]*(priceOtherD[r]-costMarginal[r]+damagedFee)*tr[d,t] for d,d2,t,r in dd2tr)
    - gp.quicksum(cstFromToD[d,d2]*tu[d,d2,t] for d,d2,t in dd2t) 
    - gp.quicksum(cstFromToD[d,d2]*td[d,d2,t] for d,d2,t in dd2t) - cstOwn*n ), GRB.MAXIMIZE)

In [ ]:
# 验证模型公式

model.write('CarRental1.lp')

# 运行优化引擎

model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 97 rows, 289 columns and 1061 nonzeros
Model fingerprint: 0x7ddb81fe
Coefficient statistics:
  Matrix range     [1e-03, 1e+00]
  Objective range  [2e+01, 7e+01]
  Bounds range     [1e+01, 3e+02]
  RHS range        [0e+00, 0e+00]
Presolve removed 49 rows and 85 columns
Presolve time: 0.01s
Presolved: 48 rows, 204 columns, 936 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.1102412e+05   3.626166e+02   0.000000e+00      0s
      69    1.2116021e+05   0.000000e+00   0.000000e+00      0s

Solved in 69 iterations and 0.01 seconds
Optimal objective  1.211602072e+05


---
## 分析

In [ ]:
# 输出报告

# 拥有的汽车总数
print(f"应拥有的最优汽车数量为: {round(n.x)}.")

# 最优利润
print(f"最优利润为: {'${:,.2f}'.format(round(model.objVal,2))}.")


The optimal number of cars to be owned is: 617.
The optimal profit is: $121,160.21.


In [ ]:
# 创建列表将日期数字标签转换为实际日期名称
dayname = ['周一','周二','周三','周四','周五','周六']

# 每天开始时各网点的未损坏汽车数量
print("\n\n_________________________________________________________________________________")
print(f"每天开始时各网点预计的未损坏汽车数量: ")
print("_________________________________________________________________________________")

undamaged_cars = pd.DataFrame(
    {
        "Day": [dayname[t] for t in days],
        "Glasgow": [round(nu['Glasgow',t].x) for t in days],
        "Manchester": [round(nu['Manchester',t].x) for t in days],
        "Birmingham": [round(nu['Birmingham',t].x) for t in days],
        "Plymouth": [round(nu['Plymouth',t].x) for t in days],
    }
)
undamaged_cars.index=[''] * len(undamaged_cars)
undamaged_cars



_________________________________________________________________________________
Estimated number of undamaged cars in depot at the beginning of each day: 
_________________________________________________________________________________


,Day,Glasgow,Manchester,Birmingham,Plymouth
,Monday,68,98,146,41
,Tuesday,66,95,155,40
,Wednesday,70,100,123,43
,Thursday,68,114,116,42
,Friday,70,102,124,43
,Saturday,67,95,158,40


In [ ]:
# 每天开始时各网点的损坏汽车数量
print("_________________________________________________________________________________")
print(f"每天开始时各网点预计的损坏汽车数量: ")
print("_________________________________________________________________________________")

damaged_cars = pd.DataFrame(
    {
        "Day": [dayname[t] for t in days],
        "Glasgow": [round(nd['Glasgow',t].x) for t in days],
        "Manchester": [round(nd['Manchester',t].x) for t in days],
        "Birmingham": [round(nd['Birmingham',t].x) for t in days],
        "Plymouth": [round(nd['Plymouth',t].x) for t in days],
    }
)
damaged_cars.index=[''] * len(damaged_cars)
damaged_cars

_________________________________________________________________________________
Estimated number of damaged cars in depot at the beginning of each day: 
_________________________________________________________________________________


,Day,Glasgow,Manchester,Birmingham,Plymouth
,Monday,8,12,20,6
,Tuesday,7,12,20,4
,Wednesday,8,13,20,5
,Thursday,8,12,21,5
,Friday,8,12,20,7
,Saturday,7,12,22,4


In [ ]:
# 每个网点每天租出的未损坏汽车数量
print("_________________________________________________________________________________")
print(f"每天从各网点租出的未损坏汽车预计数量: ")
print("_________________________________________________________________________________")

rentedOut = {}

for d in depots:
    for t in days:
        count = 0
        for d2 in depots:
            for r in rentDays:
                #print(f"Depot {d}, day {t}: cars rented out {tr[d,t].x}")
                count += pctUndamaged*pctFromToD[d,d2]*pctRent[r]*tr[d,t].x
        rentedOut[d,t] = round(count)
    

#print(rentedOut)

rentout_cars = pd.DataFrame(
    {
        "Day": [dayname[t] for t in days],
        "Glasgow": [round(rentedOut['Glasgow',t]) for t in days],
        "Manchester": [round(rentedOut['Manchester',t]) for t in days],
        "Birmingham": [round(rentedOut['Birmingham',t]) for t in days],
        "Plymouth": [round(rentedOut['Plymouth',t]) for t in days],
    }
)
rentout_cars.index=[''] * len(rentout_cars)
rentout_cars

_________________________________________________________________________________
Estimated number of undamaged cars rented out from each depot and day: 
_________________________________________________________________________________


,Day,Glasgow,Manchester,Birmingham,Plymouth
,Monday,61,89,86,37
,Tuesday,59,85,140,36
,Wednesday,63,72,111,39
,Thursday,62,103,100,38
,Friday,63,92,63,39
,Saturday,60,85,112,36


---
## 参考文献

H. Paul Williams, 《数学规划模型构建》第五版。

Copyright © 2020 Gurobi Optimization, LLC